In [1]:
import cx_Oracle as cx
import pandas as pd
import os
from loguru import logger
from sqlalchemy import types, create_engine
from risk_models.config.auth_config import ezpass_db
from risk_models.config.read_config.read_func import Read_Oracle
import datetime
import time
from sqlalchemy.types import *

os.environ['NLS_LANG'] = 'SIMPLIFIED CHINESE_CHINA.UTF8'


class Write_Oracle_Alarm(object):
    def __init__(self):
        # TODO 需要修改
        self.dmlink = ezpass_db.Authconfig.oracle_database_dbdm
        self.dwlink = ezpass_db.Authconfig.oracle_database_dbdw
        self.odslink = ezpass_db.Authconfig.oracle_database_dbods
        self.account = ezpass_db.Authconfig.ods_oracle_account
        self.write_account = ezpass_db.Authconfig.oracle_write_account_alarm
        self.write_address = ezpass_db.Authconfig.oracle_write_address_alarm
        # 目前只考虑一个conn，以后有多个写入地址再加
        self.conn = create_engine(f"oracle+cx_oracle://{self.write_account}@{self.write_address}",
                                  encoding='utf-8', convert_unicode=True)
        self.conn.autocommit = True

    def get_max_index(self, table_name):
        """
        :param table_name: 旧表表名
        :return: 返回最大的ID
        """
        read_sql = f'select max("ID") as id from {table_name.lower()}'
        # todo 这里的读取用到了dbdm是因为切换了生产环境，原本是dbods
        s_id = Read_Oracle().read_oracle(sql=read_sql, database='dbalarm')
        if s_id is None:
            return 0
        else:
            return 0 if s_id['ID'].values[0] == 'N/A' else s_id['ID'].values[0]

    def get_first_line(self, table_name):
        read_sql = f'select * from {table_name.lower()} where rownum = 1 '
        # todo 这里的读取用到了dbdm是因为切换了生产环境，原本是dbods
        old_df = Read_Oracle().read_oracle(sql=read_sql, database='dbdm')
        return old_df

    def clean_index(self, new_df, old_table_name):
        """
        清洗newdf的index
        :param new_df:
        :param old_table_name:
        :return:
        """
        new_df = new_df.reset_index()
        new_df = new_df.drop(axis=1, columns=['ID']).rename(columns={'index': 'ID'})
        max_id = self.get_max_index(old_table_name)
        if max_id is not None:
            new_df.loc[:, 'ID'] = new_df.loc[:, 'ID'].apply(lambda x: x + max_id + 1)
        else:
            pass
        new_df.set_index('ID')

        return new_df

    def add_lastupdate(self, new_df):
        now_time = datetime.datetime.now()
        now_time = datetime.datetime.strftime(now_time, format='%Y-%m-%d %H:%M:%S')
        new_df.loc[:, 'LASTUPDATE'] = now_time
        new_df.loc[:, 'LASTUPDATE'] = pd.to_datetime(new_df.loc[:, 'LASTUPDATE'], format='%Y-%m-%d %H:%M:%S')

        return new_df

    def add_iscurrent(self, new_df):
        new_df.loc[:, 'ISCURRENT'] = 1
        return new_df

    def change_lastupdate(self, old_table_name, org_code, alarm):
        """
        :param old_table_name: 表名
        :param org_code: 企业信用代码
        :param alarm: [model_code, child_mode_code] 主模块id和子模块id
        :return: None
        """
        now_time = datetime.datetime.now()
        now_time = datetime.datetime.strftime(now_time, format='%Y-%m-%d %H:%M:%S')
        if org_code is not None and alarm is None:
            try:
                self.conn.execute(
                    "update {} set LASTUPDATE =to_date('{}','yyyy-mm-dd hh24:mi:ss') where iscurrent = 1 and org_code ='{}'".format(
                        old_table_name, now_time, org_code))
            except Exception as e:
                logger.error(e)
        elif alarm is not None:
            if org_code is not None and len(alarm) == 2:
                model_code = alarm[0]
                child_mode_code = alarm[1]
                if child_mode_code == '':
                    try:
                        self.conn.execute(
                            "update {} set LASTUPDATE =to_date('{}','yyyy-mm-dd hh24:mi:ss') where iscurrent = 1 and org_code ='{}' and model_code = '{}'".format(
                                old_table_name, now_time, org_code, model_code))
                    except Exception as e:
                        logger.error(e)
                else:
                    try:
                        self.conn.execute(
                            "update {} set LASTUPDATE =to_date('{}','yyyy-mm-dd hh24:mi:ss') where iscurrent = 1 and org_code ='{}' and model_code = '{}' and child_model_code ='{}'".format(
                                old_table_name, now_time, org_code, model_code, child_mode_code))
                    except Exception as e:
                        logger.error(e)
            elif org_code is None and len(alarm) == 2:
                model_code = alarm[0]
                child_mode_code = alarm[1]
                if child_mode_code == '':
                    try:
                        self.conn.execute(
                            "update {} set LASTUPDATE =to_date('{}','yyyy-mm-dd hh24:mi:ss') where iscurrent = 1 and model_code = '{}'".format(
                                old_table_name, now_time,  model_code))
                    except Exception as e:
                        logger.error(e)
                else:
                    try:
                        self.conn.execute(
                            "update {} set LASTUPDATE =to_date('{}','yyyy-mm-dd hh24:mi:ss') where iscurrent = 1 and model_code = '{}' and child_model_code ='{}'".format(
                                old_table_name, now_time, model_code, child_mode_code))
                    except Exception as e:
                        logger.error(e)
        elif org_code is None:
            try:
                self.conn.execute(
                    "update {} set LASTUPDATE =to_date('{}','yyyy-mm-dd hh24:mi:ss') where iscurrent = 1 ".format(old_table_name, now_time))
            except Exception as e:
                logger.error(e)
        else:
            logger.warning('change_lastupdate func params error, please check params')

    def change_iscurrent(self, old_table_name, org_code, alarm):
        if org_code is not None and alarm is None:
            try:
                self.conn.execute("update {} set ISCURRENT = 0  where iscurrent = 1 and org_code ='{}'".format(old_table_name, org_code))
            except Exception as e:
                logger.error(e)
        elif alarm is not None:
            if org_code is not None and len(alarm) == 2:
                model_code = alarm[0]
                child_mode_code = alarm[1]
                if child_mode_code == '':
                    try:
                        self.conn.execute(
                            "update {} set ISCURRENT = 0  where iscurrent = 1 and org_code ='{}' and model_code = '{}'".format(old_table_name, org_code, model_code))
                    except Exception as e:
                        logger.error(e)
                else:
                    try:
                        self.conn.execute(
                            "update {} set ISCURRENT = 0  where iscurrent = 1 and org_code ='{}' and model_code = '{}' and child_model_code ='{}'".format(
                                old_table_name, org_code, model_code, child_mode_code))
                    except Exception as e:
                        logger.error(e)
            elif org_code is None and len(alarm) == 2:
                model_code = alarm[0]
                child_mode_code = alarm[1]
                if child_mode_code == '':
                    try:
                        self.conn.execute(
                            "update {} set ISCURRENT = 0 where iscurrent = 1 and model_code = '{}'".format(old_table_name, model_code))
                    except Exception as e:
                        logger.error(e)
                else:
                    try:
                        self.conn.execute(
                            "update {} set ISCURRENT = 0 where iscurrent = 1 and model_code = '{}' and child_model_code ='{}'".format(
                                old_table_name, model_code, child_mode_code))
                    except Exception as e:
                        logger.error(e)
        elif org_code is None:
            try:
                self.conn.execute("update {} set ISCURRENT = 0 where iscurrent = 1 ".format(old_table_name))
            except Exception as e:
                logger.error(e)
        else:
            logger.warning('change_iscurrent_func params error, please check params')

    @logger.catch()
    def write_oracle(self, table_name, new_df, org_code, alarm, iscurrent=True, alarm_code = None):
        """
        风控产品封装的写入接口
        :param iscurrent: 是否需要在插入的时候更新iscurrent
        :param table_name: 需要写入的表名
        :param new_df: 写入的对象dataframe
        :param org_code: 需要更新的企业信用代码
        :param alarm:  [model_code, child_mode_code] 主模块id和子模块id
        :return: None
        """
        if new_df is None:
            return
        start_time = time.time()
        new_df = self.clean_index(new_df, table_name)
        print(new_df)
        new_df = self.add_lastupdate(new_df)
        new_df = self.add_iscurrent(new_df)
        self.change_lastupdate(table_name, org_code, alarm)
        if iscurrent:
            self.change_iscurrent(table_name, org_code, alarm)
        # old_df = self.get_first_line(table_name)

        for key, value in dict(new_df.dtypes).items():
            # 如果不对object的columns进行一次astype（str）就会报错
            if value == 'object':
                new_df.loc[:, '{}'.format(key)] = new_df.loc[:, '{}'.format(key)].astype(str)

        def set_d_type_dict(df):
            type_dict = {}
            for i, j in zip(df.columns, df.dtypes):
                if "object" in str(j):
                    type_dict.update({i: VARCHAR(512)})
                if "float" in str(j):
                    type_dict.update({i: DECIMAL(20, 5)})
                if "int" in str(j):
                    type_dict.update({i: DECIMAL(20)})
            return type_dict

        logger.info('Processing... Writing {} rows into database'.format(len(new_df)))
        d_type = set_d_type_dict(new_df)
        new_df.to_sql(name=table_name.lower(), con=self.conn, if_exists='append', index=False, dtype=d_type, chunksize=2000)

        end_time = time.time()
        consum_time = end_time - start_time
        logger.info('Insert data into {} successfully! Total write time spent {}s'.format(table_name.upper(), str(consum_time)[:5]))


In [ ]:
from enum import Enum, unique
from config.param_config import params_global

@unique
class TableList(Enum):
    """
    here to save all read table for Easipass Database table reference
    all table changes must made here
    """
    if params_global.is_test:
        # 测试环境,请注意切换！！！！！！！
        # oracle-ods
        # 日志表
        BD_RISK_MODEL_LOG = 'BD_RISK_MODEL_LOG'
        #大模块分数表
        BD_RISK_CORP_SCORE_DISPLAY = 'BD_RISK_CORP_SCORE_DISPLAY'
        # 预警表
        BD_RISK_ALARM_ITEM ='BD_RISK_ALARM_ITEM'

        # 企业信息列表
        ft_gov_dtl_corp_info = 'dw_lgxc_basic.ft_gov_dtl_corp_info'
        # 涉诉信息
        MX_SHESU = 'MX_SHESU'
        # 失信信息
        MX_SHIXIN = 'MX_SHIXIN'
        # 投资信息
        MX_investor = 'MX_investor'
        # 企业法人库数据
        CORP_INFO = 'ods_zmxpq.corp_info'
        # 企业参保情况
        ZWY_DWCBQK_XXB = 'ods_zmxpq.ZWY_DWCBQK_XXB'
        # 企业水费记录
        WATER_RATE_USAGE = 'ods_zmxpq.WATER_RATE_USAGE'
        # 企业电费记录
        ELECTRIC_CHARGE_USAGE = 'ods_zmxpq.ELECTRIC_CHARGE_USAGE'
        # 企业煤气费记录
        NATURAL_GAS_USAGE = 'ods_zmxpq.NATURAL_GAS_USAGE'

        # 上飞期初库存表 - 表头
        OPENING_INVENTORY = 'OPENING_INVENTORY'
        # 上飞期初库存表 - 表体
        OPENING_INVENTORY_DETAIL = 'OPENING_INVENTORY_DETAIL'
        # 上飞库存出入库表
        EMS_STOCK_BILL = 'EMS_STOCK_BILL'
        # 上飞交货明细表
        EMS_DELIV_DETAIL = 'EMS_DELIV_DETAIL'
        # 上飞订单明细表
        EMS_ORDER_DETAIL = 'EMS_ORDER_DETAIL'
        # 上飞订单头表
        EMS_ORDER_HEAD = 'EMS_ORDER_HEAD'
        # 上飞财务表
        EMS_FINANCE_INFO = 'EMS_FINANCE_INFO'
        # 上飞生产总表
        EMS_MANUFACTURE_TOTAL = 'EMS_MANUFACTURE_TOTAL'
        # 上飞工单头表
        EMS_WORK_HEAD = 'EMS_WORK_HEAD'
        # 上飞工单耗用表
        EMS_WORK_INPUT = 'EMS_WORK_INPUT'
        # 上飞工单产出表
        EMS_WORK_OUTPUT = 'EMS_WORK_OUTPUT'
        # 上飞加工工时耗用表
        EMS_TIMECOST_INFO = 'EMS_TIMECOST_INFO'
        #综保客户订单头表（造的）
        BD_RISK_WAREHOUSE_ORDER_HEAD = 'BD_RISK_WAREHOUSE_ORDER_HEAD'
        #综保客户订单明细表（造的）
        BD_RISK_WAREHOUSE_ORDER = 'BD_RISK_WAREHOUSE_ORDER'
        #综保出入库单
        WAREHOUSE_STOCK_BILL = 'stock_bill'
        # 海关信用公示
        CUSTOMS_CREDIT = 'ODS_ZMXPQ.CUSTOMS_CREDIT'
        # 企业信息表
        BD_RISK_CORP_INFO_BASIC = 'BD_RISK_CORP_INFO_BASIC'

        # 首页统计用到的表
        BILL_DIR_BSC = 'ODS_ZMXPQ.BILL_DIR_BSC'
        BILL_DIR_DT = 'ODS_ZMXPQ.BILL_DIR_DT'
        BILL_DIR_EXP_BSC = 'ODS_ZMXPQ.BILL_DIR_EXP_BSC'
        BILL_DIR_EXP_DT = 'ODS_ZMXPQ.BILL_DIR_EXP_DT'
        MX_DISPLAY_JINGYU = 'BD_RISK_DISPLAY_JINGYU'
        MX_DISPLAY_TAX = 'BD_RISK_DISPLAY_TAX'

        # # 模型数据
        # ST1明细表&结果表
        BD_RISK_DETAIL_STOCK_ST1 = 'BD_RISK_DETAIL_STOCK_ST1'
        BD_RISK_RESULT_STOCK_ST1 = 'BD_RISK_RESULT_STOCK_ST1'
        # ST2明细表&结果表
        BD_RISK_DETAIL_STOCK_ST2 = 'BD_RISK_DETAIL_STOCK_ST2'
        BD_RISK_RESULT_STOCK_ST2 = 'BD_RISK_RESULT_STOCK_ST2'
        # ST3明细表&结果表
        BD_RISK_DETAIL_STOCK_ST3 = 'BD_RISK_DETAIL_STOCK_ST3'
        BD_RISK_RESULT_STOCK_ST3 = 'BD_RISK_RESULT_STOCK_ST3'
        # ST4 明细表&结果表
        BD_RISK_DETAIL_STOCK_ST4 = 'BD_RISK_DETAIL_STOCK_ST4'
        BD_RISK_RESULT_STOCK_ST4 = 'BD_RISK_RESULT_STOCK_ST4'
        # ST5 明细表&结果表
        BD_RISK_DETAIL_STOCK_ST5 = 'BD_RISK_DETAIL_STOCK_ST5'
        BD_RISK_RESULT_STOCK_ST5 = 'BD_RISK_RESULT_STOCK_ST5'

        # FN1 明细表&结果表
        BD_RISK_DETAIL_FINANCE_FN1 = 'BD_RISK_DETAIL_FINANCE_FN1'
        BD_RISK_RESULT_FINANCE_FN1 = 'BD_RISK_RESULT_FINANCE_FN1'
        # FN2 明细表&结果表
        BD_RISK_DETAIL_FINANCE_FN2 = 'BD_RISK_DETAIL_FINANCE_FN2'
        BD_RISK_RESULT_FINANCE_FN2 = 'BD_RISK_RESULT_FINANCE_FN2'

        # PD1 明细表&结果表
        BD_RISK_DETAIL_PRODUCTION_PD1 = 'BD_RISK_DETAIL_PRODUCTION_PD1'
        BD_RISK_RESULT_PRODUCTION_PD1 = 'BD_RISK_RESULT_PRODUCTION_PD1'
        # PD2 明细表&结果表
        BD_RISK_DETAIL_PRODUCTION_PD2 = 'BD_RISK_DETAIL_PRODUCTION_PD2'
        BD_RISK_RESULT_PRODUCTION_PD2 = 'BD_RISK_RESULT_PRODUCTION_PD2'
        # PD3 明细表&结果表
        BD_RISK_DETAIL_PRODUCTION_PD3 = 'BD_RISK_DETAIL_PRODUCTION_PD3'
        BD_RISK_RESULT_PRODUCTION_PD3 = 'BD_RISK_RESULT_PRODUCTION_PD3'
        # PD4 明细表&结果表
        BD_RISK_DETAIL_PRODUCTION_PD4 = 'BD_RISK_DETAIL_PRODUCTION_PD4'
        BD_RISK_RESULT_PRODUCTION_PD4 = 'BD_RISK_RESULT_PRODUCTION_PD4'

        # SC1 明细表&结果表
        BD_RISK_DETAIL_SUPPLYCHAIN_SC1 = 'BD_RISK_DETAIL_SUPPLYCHAIN_SC1'
        BD_RISK_RESULT_SUPPLYCHAIN_SC1 = 'BD_RISK_RESULT_SUPPLYCHAIN_SC1'

        # WH1 明细表&结果表
        BD_RISK_DETAIL_WAREHOUSE_WH1_C = 'BD_RISK_DETAIL_WAREHOUSE_WH1_C'
        BD_RISK_DETAIL_WAREHOUSE_WH1_S = 'BD_RISK_DETAIL_WAREHOUSE_WH1_S'
        BD_RISK_RESULT_WAREHOUSE_WH1 = 'BD_RISK_RESULT_WAREHOUSE_WH1'
        # WH2 明细表&结果表
        BD_RISK_DETAIL_WAREHOUSE_WH2 = 'BD_RISK_DETAIL_WAREHOUSE_WH2'
        BD_RISK_RESULT_WAREHOUSE_WH2 = 'BD_RISK_RESULT_WAREHOUSE_WH2'

        # CR1 明细表&结果表
        BD_RISK_DETAIL_CREDIT_CR1 = 'BD_RISK_DETAIL_CREDIT_CR1'
        BD_RISK_RESULT_CREDIT_CR1 = 'BD_RISK_RESULT_CREDIT_CR1'
        # CR2 明细表&结果表
        BD_RISK_DETAIL_CREDIT_CR2 = 'BD_RISK_DETAIL_CREDIT_CR2'
        BD_RISK_RESULT_CREDIT_CR2 = 'BD_RISK_RESULT_CREDIT_CR2'
        # CR3 明细表&结果表
        BD_RISK_DETAIL_CREDIT_CR3 = 'BD_RISK_DETAIL_CREDIT_CR3'
        BD_RISK_RESULT_CREDIT_CR3 = 'BD_RISK_RESULT_CREDIT_CR3'

        # # TR1 信息表, 结果表
        BD_RISK_TRACK_INFO = 'BD_RISK_TRACK_INFO'
        BD_RISK_RESULT_TRACK_TR1 = 'BD_RISK_RESULT_TRACK_TR1'
        ## TD2 结果表
        BD_RISK_RESULT_TRADE_TD2 = 'BD_RISK_RESULT_TRADE_TD2'
        ## TD1 结果表
        BD_RISK_RESULT_TRADE_TD1 = 'BD_RISK_RESULT_TRADE_TD1'

        # # 亿通数据（测试环境暂无对应表）
        # 进口海运报关单表头
        FT_I_DTL_SEA_PRE_RECORDED = 'DW_I_BASIC.FT_I_DTL_SEA_PRE_RECORDED'
        # 进口其他报关单表头
        FT_I_DTL_OTR_PRE_RECORDED = 'DW_I_BASIC.FT_I_DTL_OTR_PRE_RECORDED'
        # 进口海运报关单商品信息表
        FT_I_DTL_SEA_LIST = 'DW_I_BASIC.FT_I_DTL_SEA_LIST'
        # 进口其他报关单商品信息表
        FT_I_DTL_OTR_LIST = 'DW_I_BASIC.FT_I_DTL_OTR_LIST'
        # 进口海运箱单
        FT_I_DTL_SEA_CONTAINER = 'DW_I_BASIC.FT_I_DTL_SEA_CONTAINER'
        # 进口海运COARRI
        FT_I_DTL_COARRI_CTNR = 'DW_I_BASIC.FT_I_DTL_COARRI_CTNR'

        # 出口海运报关单表头
        FT_E_DTL_SEA_PRE_RECORDED = 'DW_E_BASIC.FT_E_DTL_SEA_PRE_RECORDED'
        # 出口其他报关单表头
        FT_E_DTL_OTR_PRE_RECORDED = 'DW_E_BASIC.FT_E_DTL_OTR_PRE_RECORDED'
        # 出口海运报关单商品信息表
        FT_E_DTL_SEA_LIST = 'DW_E_BASIC.FT_E_DTL_SEA_LIST'
        # 出口其他报关单商品信息表
        FT_E_DTL_OTR_LIST = 'DW_E_BASIC.FT_E_DTL_OTR_LIST'
        # 出口海运箱子
        FT_E_DTL_SEA_CONTAINER = 'DW_E_BASIC.FT_E_DTL_SEA_CONTAINER'
        # 出口海运COARRI
        FT_E_DTL_COARRI_CTNR = 'DW_E_BASIC.FT_E_DTL_COARRI_CTNR'

        FT_I_DTL_TAX_INFO = 'DW_I_BASIC.FT_I_DTL_TAX_INFO'
        FT_E_DTL_TAX_INFO = 'DW_E_BASIC.FT_E_DTL_TAX_INFO'

        # 承运人码表
        DIM_OPERATOR = 'DIM.DIM_OPERATOR'
        # 企业列表（洋山围网）
        DIM_FTZ_CORP = 'DIM.DIM_FTZ_CORP'
        # 报关单经营单位信息表
        DIM_TRADER = 'DIM.DIM_TRADER'
        # 国家代码码表
        DIM_COUNTRY = 'DIM.DIM_COUNTRY'
        # 币制码表
        MAP_GJ_CURRENCY = 'DIM.MAP_GJ_CURRENCY'

        # # # BVD数据
        MX_BVD = 'MX_BVD'
        # 进出口报关单统计表
        FT_CUS_DWS_TRADE = 'DW_STA.FT_CUS_DWS_TRADE'
        # 指标货类表
        FT_STA_GOODSOWNER_MAIN_CLASS = 'DW_STA.FT_STA_GOODSOWNER_MAIN_CLASS'
        # 企业oneid表
        DW_CORP_CUSDEC = 'dw_corp_basic.dw_corp_cusdec'
        # 报关单表
        FT_CUS_DWS_ENTRY = 'DW_STA.FT_CUS_DWS_ENTRY'

    else:
        # 生产环境
        # oracle-ods
        # 日志表
        BD_RISK_MODEL_LOG = 'BD_RISK_MODEL_LOG'
        # 大模块分数表
        BD_RISK_CORP_SCORE_DISPLAY = 'BD_RISK_CORP_SCORE_DISPLAY'
        # 预警表
        BD_RISK_ALARM_ITEM ='BD_RISK_ALARM_ITEM'

        # # 亿通数据（测试环境暂无对应表）
        # 进口海运报关单表头
        FT_I_DTL_SEA_PRE_RECORDED = 'DW_I_BASIC.FT_I_DTL_SEA_PRE_RECORDED'
        # 进口其他报关单表头
        FT_I_DTL_OTR_PRE_RECORDED = 'DW_I_BASIC.FT_I_DTL_OTR_PRE_RECORDED'
        # 进口海运报关单商品信息表
        FT_I_DTL_SEA_LIST = 'DW_I_BASIC.FT_I_DTL_SEA_LIST'
        # 进口其他报关单商品信息表
        FT_I_DTL_OTR_LIST = 'DW_I_BASIC.FT_I_DTL_OTR_LIST'
        # 进口海运箱单
        FT_I_DTL_SEA_CONTAINER = 'DW_I_BASIC.FT_I_DTL_SEA_CONTAINER'
        # 进口海运COARRI
        FT_I_DTL_COARRI_CTNR = 'DW_I_BASIC.FT_I_DTL_COARRI_CTNR'

        # 出口海运报关单表头
        FT_E_DTL_SEA_PRE_RECORDED = 'DW_E_BASIC.FT_E_DTL_SEA_PRE_RECORDED'
        # 出口其他报关单表头
        FT_E_DTL_OTR_PRE_RECORDED = 'DW_E_BASIC.FT_E_DTL_OTR_PRE_RECORDED'
        # 出口海运报关单商品信息表
        FT_E_DTL_SEA_LIST = 'DW_E_BASIC.FT_E_DTL_SEA_LIST'
        # 出口其他报关单商品信息表
        FT_E_DTL_OTR_LIST = 'DW_E_BASIC.FT_E_DTL_OTR_LIST'
        # 出口海运箱子
        FT_E_DTL_SEA_CONTAINER = 'DW_E_BASIC.FT_E_DTL_SEA_CONTAINER'
        # 出口海运COARRI
        FT_E_DTL_COARRI_CTNR = 'DW_E_BASIC.FT_E_DTL_COARRI_CTNR'

        # 进出口报关单统计表
        FT_CUS_DWS_TRADE = 'DW_STA.FT_CUS_DWS_TRADE'
        # 进出口海关税单
        FT_I_DTL_TAX_INFO = 'DW_I_BASIC.FT_I_DTL_TAX_INFO'
        FT_E_DTL_TAX_INFO = 'DW_E_BASIC.FT_E_DTL_TAX_INFO'
        # 指标货类表
        FT_STA_GOODSOWNER_MAIN_CLASS = 'DW_STA.FT_STA_GOODSOWNER_MAIN_CLASS'
        # 企业oneid表
        DW_CORP_CUSDEC = 'dw_corp_basic.dw_corp_cusdec'
        # 报关单表
        FT_CUS_DWS_ENTRY = 'DW_STA.FT_CUS_DWS_ENTRY'
        # 承运人码表
        DIM_OPERATOR = 'DIM.DIM_OPERATOR'
        # 企业列表（洋山围网）
        DIM_FTZ_CORP = 'DIM.DIM_FTZ_CORP'
        # 报关单经营单位信息表
        DIM_TRADER = 'DIM.DIM_TRADER'
        # 国家代码码表
        DIM_COUNTRY = 'DIM.DIM_COUNTRY'
        # 币制码表
        MAP_GJ_CURRENCY = 'DIM.MAP_GJ_CURRENCY'

        # 企业信息列表
        ft_gov_dtl_corp_info = 'dw_lgxc_basic.ft_gov_dtl_corp_info'
        # 涉诉信息
        MX_SHESU = 'MX_SHESU'
        # 失信信息
        MX_SHIXIN = 'MX_SHIXIN'
        # 投资信息
        MX_investor = 'MX_investor'
        # 企业法人库数据
        CORP_INFO = 'ods_zmxpq.corp_info'
        # 企业参保情况
        ZWY_DWCBQK_XXB = 'ods_zmxpq.ZWY_DWCBQK_XXB'
        # 企业水费记录
        WATER_RATE_USAGE = 'ods_zmxpq.WATER_RATE_USAGE'
        # 企业电费记录
        ELECTRIC_CHARGE_USAGE = 'ods_zmxpq.ELECTRIC_CHARGE_USAGE'
        # 企业煤气费记录
        NATURAL_GAS_USAGE = 'ods_zmxpq.NATURAL_GAS_USAGE'
        # 海关信用公示
        CUSTOMS_CREDIT = 'ODS_ZMXPQ.CUSTOMS_CREDIT'
        # 企业信息表
        BD_RISK_CORP_INFO_BASIC = 'BD_RISK.BD_RISK_CORP_INFO_BASIC'
        # # BVD数据
        MX_BVD = 'MX_BVD'

        # # 企业ERP数据
        # 上飞期初库存表 - 表头
        OPENING_INVENTORY = 'OPENING_INVENTORY'
        # 上飞期初库存表 - 表体
        OPENING_INVENTORY_DETAIL = 'OPENING_INVENTORY_DETAIL'
        # 上飞库存出入库表
        EMS_STOCK_BILL = 'ODS_ZMXPQ.EMS_STOCK_BILL'
        # 上飞交货明细表
        EMS_DELIV_DETAIL = 'ods_zmxpq.EMS_DELIV_DETAIL'
        # 上飞订单明细表
        EMS_ORDER_DETAIL = 'ods_zmxpq.EMS_ORDER_DETAIL'
        # 上飞订单头表
        EMS_ORDER_HEAD = 'ods_zmxpq.EMS_ORDER_HEAD'
        # 上飞财务表
        EMS_FINANCE_INFO = 'ods_zmxpq.EMS_FINANCE_INFO'
        # 上飞生产总表
        EMS_MANUFACTURE_TOTAL = 'ods_zmxpq.EMS_MANUFACTURE_TOTAL'
        # 上飞工单头表
        EMS_WORK_HEAD = 'ods_zmxpq.EMS_WORK_HEAD'
        # 上飞工单耗用表
        EMS_WORK_INPUT = 'ods_zmxpq.EMS_WORK_INPUT'
        # 上飞工单产出表
        EMS_WORK_OUTPUT = 'ods_zmxpq.EMS_WORK_OUTPUT'
        # 上飞加工工时耗用表
        EMS_TIMECOST_INFO = 'ods_zmxpq.EMS_TIMECOST_INFO'
        # 综保客户订单头表（造的）
        BD_RISK_WAREHOUSE_ORDER_HEAD = 'BD_RISK_WAREHOUSE_ORDER_HEAD'
        # 综保客户订单明细表（造的）
        BD_RISK_WAREHOUSE_ORDER = 'BD_RISK_WAREHOUSE_ORDER'
        # 综保出入库单
        WAREHOUSE_STOCK_BILL = 'ods_zmxpq.stock_bill'

        # 首页统计用到的表
        BILL_DIR_BSC = 'ODS_ZMXPQ.BILL_DIR_BSC'
        BILL_DIR_DT = 'ODS_ZMXPQ.BILL_DIR_DT'
        BILL_DIR_EXP_BSC = 'ODS_ZMXPQ.BILL_DIR_EXP_BSC'
        BILL_DIR_EXP_DT = 'ODS_ZMXPQ.BILL_DIR_EXP_DT'
        MX_DISPLAY_JINGYU = 'BD_RISK_DISPLAY_JINGYU'
        MX_DISPLAY_TAX = 'BD_RISK_DISPLAY_TAX'


        # # 模型数据
        # ST1明细表&结果表
        BD_RISK_DETAIL_STOCK_ST1 = 'BD_RISK_DETAIL_STOCK_ST1'
        BD_RISK_RESULT_STOCK_ST1 = 'BD_RISK_RESULT_STOCK_ST1'
        # ST2明细表&结果表
        BD_RISK_DETAIL_STOCK_ST2 = 'BD_RISK_DETAIL_STOCK_ST2'
        BD_RISK_RESULT_STOCK_ST2 = 'BD_RISK_RESULT_STOCK_ST2'
        # ST3明细表&结果表
        BD_RISK_DETAIL_STOCK_ST3 = 'BD_RISK_DETAIL_STOCK_ST3'
        BD_RISK_RESULT_STOCK_ST3 = 'BD_RISK_RESULT_STOCK_ST3'
        # ST4 明细表&结果表
        BD_RISK_DETAIL_STOCK_ST4 = 'BD_RISK_DETAIL_STOCK_ST4'
        BD_RISK_RESULT_STOCK_ST4 = 'BD_RISK_RESULT_STOCK_ST4'
        # ST5 明细表&结果表
        BD_RISK_DETAIL_STOCK_ST5 = 'BD_RISK_DETAIL_STOCK_ST5'
        BD_RISK_RESULT_STOCK_ST5 = 'BD_RISK_RESULT_STOCK_ST5'

        # FN1 明细表&结果表
        BD_RISK_DETAIL_FINANCE_FN1 = 'BD_RISK_DETAIL_FINANCE_FN1'
        BD_RISK_RESULT_FINANCE_FN1 = 'BD_RISK_RESULT_FINANCE_FN1'
        BD_RISK_GRAPH_FINANCE_FN1 = 'BD_RISK_GRAPH_FINANCE_FN1'
        # FN2 明细表&结果表
        BD_RISK_DETAIL_FINANCE_FN2 = 'BD_RISK_DETAIL_FINANCE_FN2'
        BD_RISK_RESULT_FINANCE_FN2 = 'BD_RISK_RESULT_FINANCE_FN2'

        # PD1 明细表&结果表
        BD_RISK_DETAIL_PRODUCTION_PD1 = 'BD_RISK_DETAIL_PRODUCTION_PD1'
        BD_RISK_RESULT_PRODUCTION_PD1 = 'BD_RISK_RESULT_PRODUCTION_PD1'
        # PD2 明细表&结果表
        BD_RISK_DETAIL_PRODUCTION_PD2 = 'BD_RISK_DETAIL_PRODUCTION_PD2'
        BD_RISK_RESULT_PRODUCTION_PD2 = 'BD_RISK_RESULT_PRODUCTION_PD2'
        # PD3 明细表&结果表
        BD_RISK_DETAIL_PRODUCTION_PD3 = 'BD_RISK_DETAIL_PRODUCTION_PD3'
        BD_RISK_RESULT_PRODUCTION_PD3 = 'BD_RISK_RESULT_PRODUCTION_PD3'
        # PD4 明细表&结果表
        BD_RISK_DETAIL_PRODUCTION_PD4 = 'BD_RISK_DETAIL_PRODUCTION_PD4'
        BD_RISK_RESULT_PRODUCTION_PD4 = 'BD_RISK_RESULT_PRODUCTION_PD4'

        # SC1 明细表&结果表
        BD_RISK_DETAIL_SUPPLYCHAIN_SC1 = 'BD_RISK_DETAIL_SUPPLYCHAIN_SC1'
        BD_RISK_RESULT_SUPPLYCHAIN_SC1 = 'BD_RISK_RESULT_SUPPLYCHAIN_SC1'

        # WH1 明细表&结果表
        BD_RISK_DETAIL_WAREHOUSE_WH1_C = 'BD_RISK_DETAIL_WAREHOUSE_WH1_C'
        BD_RISK_DETAIL_WAREHOUSE_WH1_S = 'BD_RISK_DETAIL_WAREHOUSE_WH1_S'
        BD_RISK_RESULT_WAREHOUSE_WH1 = 'BD_RISK_RESULT_WAREHOUSE_WH1'
        # WH2 明细表&结果表
        BD_RISK_DETAIL_WAREHOUSE_WH2 = 'BD_RISK_DETAIL_WAREHOUSE_WH2'
        BD_RISK_RESULT_WAREHOUSE_WH2 = 'BD_RISK_RESULT_WAREHOUSE_WH2'

        # CR1
        BD_RISK_DETAIL_CREDIT_CR1 = 'BD_RISK_DETAIL_CREDIT_CR1'
        BD_RISK_RESULT_CREDIT_CR1 = 'BD_RISK_RESULT_CREDIT_CR1'
        # CR2
        BD_RISK_DETAIL_CREDIT_CR2 = 'BD_RISK_DETAIL_CREDIT_CR2'
        BD_RISK_RESULT_CREDIT_CR2 = 'BD_RISK_RESULT_CREDIT_CR2'
        # CR3 明细表&结果表
        BD_RISK_DETAIL_CREDIT_CR3 = 'BD_RISK_DETAIL_CREDIT_CR3'
        BD_RISK_RESULT_CREDIT_CR3 = 'BD_RISK_RESULT_CREDIT_CR3'

        # # TR1 信息表, 结果表
        BD_RISK_TRACK_INFO = 'BD_RISK_TRACK_INFO'
        BD_RISK_RESULT_TRACK_TR1 = 'BD_RISK_RESULT_TRACK_TR1'

        ## TD2 结果表
        BD_RISK_RESULT_TRADE_TD2 = 'BD_RISK_RESULT_TRADE_TD2'
        ## TD1 结果表
        BD_RISK_RESULT_TRADE_TD1 = 'BD_RISK_RESULT_TRADE_TD1'

        #公共的电费与社保结果表 旧风控用
        MX_PUBLIC_ELECTRIC = 'BD_RISK.MX_PUBLIC_ELECTRIC'
        MX_PUBLIC_INSURANCE = 'BD_RISK.MX_PUBLIC_INSURANCE'



In [4]:


from config.write_config.write_func import Write_Oracle
import pandas as pd
import datetime


class DisplayDp2(object):
    def __init__(self, base_time, child_task_id):
        self.model_code = 'DISPLAY'
        self.child_model_code = 'DP2'
        self.child_task_id = child_task_id
        self.org_code = None
        # Risk_logger(child_task_id=self.child_task_id, exec_status=None)

        # 参数读取
        # startdt 是 base_time - 730 days，enddt为base_time
        self.startdt = datetime.datetime.strptime(base_time,"%Y-%m-%d %H:%M:%S") + datetime.timedelta(days=-730)
        self.startdt = self.startdt.strftime('%Y-%m-%d')
        self.enddt = datetime.datetime.strptime(base_time,"%Y-%m-%d %H:%M:%S").strftime('%Y-%m-%d')


    def display_dp2(self):
        def date_standard(df):
            """
            des: 填充缺失的时间
            input df['col']
            output df_series
            """
            b=[]
            # date_begin=datetime.datetime(2020,10,1)
            # date_end=datetime.datetime(2021,11,1)
            date_begin=df.min()
            date_end=df.max()
            a=pd.date_range(date_begin,date_end,freq='MS',normalize=True)
            for i in a:
                i=str(i)
                #     print(datetime.datetime.strptime(str(i),'%Y-%m-%d %H:%M:%S'))
                #     i=i.replace('31','01')
                #     i=i.replace('30','01')
                #     i=i.replace('29','01')
                #     i=i.replace('28','01')
                i=datetime.datetime.strptime(i,'%Y-%m-%d %H:%M:%S')
                b.append(i)
            date_serie=pd.DataFrame(b)
            date_serie=date_serie.rename(columns={0:'STANDARD_DATE'})
            return date_serie

        #     径予进口
        sql=f'''SELECT 'I' AS IE_FLAG,to_date(TO_CHAR(a.DCL_TIME,'YYYYMM'),'yyyymm') AS DATETIME, \
        COUNT(DISTINCT A.ORDER_NO) AS ENTRY_NUM, COUNT(DISTINCT A.BIZOP_ETPS_NO) AS TRADE_NUM, \
        SUM(B.DCL_TOTAL_AMT) AS MONEY,'径予' as type \
        FROM {TableList.BILL_DIR_BSC.value} A JOIN {TableList.BILL_DIR_DT.value} B \
        ON A.ID =B.PID WHERE A.DCL_TIME >= DATE '{self.startdt}' AND A.DCL_TIME< DATE'{self.enddt}' \
        GROUP BY to_date(TO_CHAR(a.DCL_TIME,'YYYYMM'),'yyyymm') \
        '''
        t3=Read_Oracle().read_oracle(sql=sql,database='dbods')

        # 径予出口
        sql=f'''\
        SELECT 'E' AS IE_FLAG,TO_DATE(TO_CHAR(A.DCL_TIME,'YYYYMM'),'yyyymm') AS DATETIME, \
        COUNT(DISTINCT A.ORDER_NO) AS ENTRY_NUM, COUNT(DISTINCT A.BIZOP_ETPS_NO) AS TRADE_NUM, \
        SUM(B.DCL_TOTAL_AMT) AS MONEY,'径予' as type \
        FROM {TableList.BILL_DIR_EXP_BSC.value} A JOIN {TableList.BILL_DIR_EXP_DT.value} B ON A.ID =B.PID \
        WHERE A.DCL_TIME >= DATE'{self.startdt}' AND A.DCL_TIME< DATE'{self.enddt}' \
        GROUP BY TO_DATE(TO_CHAR(A.DCL_TIME,'YYYYMM'),'yyyymm')'''
        t4=Read_Oracle().read_oracle(sql=sql,database='dbods')

        df_jingyu=pd.concat([t3,t4])
        df_jingyu.reset_index(inplace=True,drop=True)
        df_jingyu=df_jingyu.fillna(0)

        date_serie=date_standard(df_jingyu['DATETIME'])

        df_final_I=date_serie.merge(df_jingyu[df_jingyu['IE_FLAG']=='I'],left_on='STANDARD_DATE',right_on='DATETIME',how='left')
        df_final_I['IE_FLAG']='I';df_final_I['TYPE']='径予';df_final_I=df_final_I.fillna(0)
        df_final_E=date_serie.merge(df_jingyu[df_jingyu['IE_FLAG']=='E'],left_on='STANDARD_DATE',right_on='DATETIME',how='left')
        df_final_E['IE_FLAG']='E';df_final_E['TYPE']='径予';df_final_E=df_final_E.fillna(0)
        df_final=pd.concat([df_final_I,df_final_E],axis=0,ignore_index=True)
        df_final=df_final.drop(['DATETIME'],axis=1)
        df_final=df_final.rename(columns={'STANDARD_DATE':'DATETIME'})

        now = datetime.datetime.strftime(datetime.datetime.now(), format='%Y-%m-%d %H:%M:%S')
        df_final['CHECK_TIME']=now
        df_final['CHECK_TIME']= pd.to_datetime(df_final['CHECK_TIME'], format='%Y-%m-%d %H:%M:%S')
        df_final = df_final.reset_index().rename(columns = {'index':'ID'})
        # df_final['ORG_CODE'] = df_final['ORG_CODE'].astype(str)
        df_final['CHECK_TIME'] = df_final['CHECK_TIME'].astype('datetime64')
        df_final['DATETIME'] = df_final['DATETIME'].astype('datetime64')
        df_final['IE_FLAG'] = df_final['IE_FLAG'].astype(str)
        df_final['ENTRY_NUM'] = df_final['ENTRY_NUM'].astype(float)
        df_final['TRADE_NUM'] = df_final['TRADE_NUM'].astype(float)
        df_final['MONEY'] = df_final['MONEY'].astype(float)
        df_final['TYPE'] = df_final['TYPE'].astype(str)

        # 写入
        # Write_Oracle().write_oracle(f'{TableList.MX_DISPLAY_JINGYU.value}',df_final,org_code=None,alarm=None)


NameError: name 'TableList' is not defined

In [ ]:
import cx_Oracle
from enum import unique
class params_global():
    is_test = "True"
    # is_test = "False"

    nodata_exec_status = 2
"""
here to save all read config for Ezpass database
"""
class ezpass_db:
    # @unique
    class Authconfig:
        if params_global.is_test:
            # 测试环境cusrc
            ods_oracle_account = 'CUSRC/easipass'
            oracle_database_dbods = '192.168.129.149:1521/test12c'
            oracle_write_account = 'CUSRC:easipass'
            oracle_write_address = '192.168.129.149:1521/?service_name=test12c'
            oracle_write_account_alarm = 'DW_CUS_RC:easipass'
            oracle_write_address_alarm = '192.168.130.225:1521/?service_name=pdbcusdev'

            # # 测试环境 dw
            dw_oracle_account = 'BI_OPER/easipass'
            oracle_database_dbdw = '192.168.129.179:1521/testdw'

            # 测试环境BD_RISK
            ods_oracle_account = 'BD_RISK/easipass'
            oracle_database_dbods = '192.168.129.149:1521/test12c'
            # oracle_write_account = 'BD_RISK:easipass'
            # oracle_write_address = '192.168.129.149:1521/?service_name=test12c'

            dm_oracle_account = 'BD_RISK/infa4A9F'
            oracle_database_dbdm = '192.168.132.52:1521/dbdm'

            gold_oracle_account = 'n_epz_cus/n_epz_cus'
            oracle_database_gold = '192.168.131.125:1521/zmqwdb'

            alarm_oracle_account = 'DW_CUS_RC/easipass'
            oracle_database_alarm = '192.168.130.225:1521/pdbcusdev'

            lgsa_oracle_account = 'lgsa/lgsa'
            oracle_database_lgsa = '192.168.131.125:1521/zmqwdb'

            testlog_oracle_account = 'BD_RISK/testinfaer'
            oracle_database_testlog = '192.168.129.163:1521/TESTDMAPP'
        else:
            # 生产环境 CUSRC
            # ods_oracle_account = 'CUSRC/infa5F1D'
            # oracle_database_dbods = '192.168.132.12:1521/dbods'
            # oracle_write_account = 'CUSRC:infa5F1D'
            # oracle_write_address = '192.168.132.12:1521/?service_name=dbods'

            # 生产环境 BD_RISK
            oracle_write_account = 'BD_RISK:infa4A9F'
            oracle_write_address = '192.168.132.52:1521/?service_name=dbdm'
            # dbods
            ods_oracle_account = 'CUSRC/infa5F1D'
            oracle_database_dbods = '192.168.132.12:1521/dbods'
            # db_dw
            dw_oracle_account = 'BI_OPER/infa6D83'
            oracle_database_dbdw = '192.168.132.17:1521/dbdw'
            # db_dm
            dm_oracle_account = 'BD_RISK/infa4A9F'
            oracle_database_dbdm = '192.168.132.52:1521/dbdm'

            # Mongodb_config
            mongo_read_oper = 'ods_sens_oper:mgoD52F'
            mongo_ip = '192.168.132.74:27017'
            mongo_database = 'ods_sens_zmxpq'
            mongo_table = 'credit'
# from loguru import logger
import time
import pandas as pd
class Read_Oracle(object):
    def __init__(self):
        self.link = None
        self.account = None

    def read_oracle_df(self, sql):
        print(self.account)
        print(self.link)
        conn = cx.connect(self.account+'@'+self.link, encoding='utf8')
        data_gen = pd.read_sql(con=conn, sql=sql, chunksize=100000)
        gen_list = []
        for gen in data_gen:
            gen_list.append(gen)
        if len(gen_list) ==0:
            print('Sql cannot find table, please check database')
            data_df = None
        else:
            data_df = pd.concat(gen_list)
        return data_df


    def read_oracle(self, sql, database):
        start_time = time.time()
        """
        :arg
            :param sql : 读取的select 脚本
            :param databse ：目标database ； dbods、dbdw、dbdm
        :return
            返回查询的结果dataframe
        """
        sql = sql
        if database == 'dbods':
            self.link = ezpass_db.Authconfig.oracle_database_dbods
            self.account = ezpass_db.Authconfig.ods_oracle_account
        elif database == 'dbdw':
            self.link = ezpass_db.Authconfig.oracle_database_dbdw
            self.account = ezpass_db.Authconfig.dw_oracle_account
        elif database == 'dbdm':
            self.link = ezpass_db.Authconfig.oracle_database_dbdm
            self.account = ezpass_db.Authconfig.dm_oracle_account
        elif database == 'dbgold':
            self.link = ezpass_db.Authconfig.oracle_database_gold
            self.account = ezpass_db.Authconfig.gold_oracle_account
        elif database == 'dbalarm':
            self.link = ezpass_db.Authconfig.oracle_database_alarm
            self.account = ezpass_db.Authconfig.alarm_oracle_account
        elif database == 'dblgsa':
            self.link = ezpass_db.Authconfig.oracle_database_lgsa
            self.account = ezpass_db.Authconfig.lgsa_oracle_account
        elif database == 'testlog':
            self.link = ezpass_db.Authconfig.oracle_database_testlog
            self.account = ezpass_db.Authconfig.testlog_oracle_account
        else:
            print('read_oracle config error, please match your input sql and db')
            return

        data_df = self.read_oracle_df(sql)
        if data_df is None:
            data_df = pd.DataFrame()
        if data_df is not None:
            data_df = data_df.fillna('N/A')
        end_time = time.time()
        consum_time = end_time - start_time
        print('Read Table successfully! , Total read time spent {}s'.format(str(consum_time)[:5]))
        return data_df
def deal_context_dic(df):
    temp_dic = {}
    str = "{"
    temp_dic['businessStartTime'] = datetime.datetime.now()
    temp_dic['businessEndTime'] = datetime.datetime.now()
    temp_dic['entry'] = df['entry']
    for key, value in temp_dic.items():
        str += "\"%s\":\"%s\"," % (key, value)
    str = str[:-1]
    return str + "}"
def deal_entry_dic(df):
    temp_dic = {}
    str = "{"
    temp_dic['entryId'] = df['ENTRY_ID']
    temp_dic['entryIdCompare'] = df['ENTRY_ID_COMPARE']
    temp_dic['comparison'] = df['COMPARISON']
    temp_dic['ownerCode'] = df['CONSIGN_SCC']
    temp_dic['ownerName'] = df['CONSIGN_NAME']
    temp_dic['ownerCodeCompare'] = df['CONSIGN_SCC_COMPARE']
    temp_dic['ownerNameCompare'] = df['CONSIGN_NAME_COMPARE']
    for key, value in temp_dic.items():
        str += "\"%s\":\"%s\"," % (key, value)
    str = str[:-1]
    return str + "},"
def deal_comparison_dic(df):
    temp_dic = {}
    str = "{"
    temp_dic['code_t'] = df['G_NAME']
    temp_dic['qty1'] = df['G_QTY']
    temp_dic['originCountry'] = df['ORIGIN_COUNTRY']
    temp_dic['codeTCompare'] = df['G_NAME_COMPARE']
    temp_dic['qty1Compare'] = df['G_QTY_COMPARE']
    temp_dic['originCountryCompare'] = df['ORIGIN_COUNTRY_COMPARE']
    for key, value in temp_dic.items():
        str += "\"%s\":\"%s\"," % (key, value)
    str = str[:-1]
    return str + "},"
import cx_Oracle as cx
import pandas as pd
import os
import datetime
from loguru import logger
import time
pd.set_option('display.max_columns', None)
from config.auth_config import ezpass_db
os.environ['NLS_LANG'] = 'SIMPLIFIED CHINESE_CHINA.UTF8'
def deal_context_dic(df):
    temp_dic = {}
    str = "{"
    temp_dic['businessStartTime'] = datetime.datetime.now()
    temp_dic['businessEndTime'] = datetime.datetime.now()
    temp_dic['entry'] = df['entry']
    for key, value in temp_dic.items():
        str += "\"%s\":\"%s\"," % (key, value)
    str = str[:-1]
    return str + "}"
def deal_entry_dic(df):
    temp_dic = {}
    str = "{"
    temp_dic['entryId'] = df['ENTRY_ID']
    temp_dic['entryIdCompare'] = df['ENTRY_ID_COMPARE']
    temp_dic['comparison'] = df['COMPARISON']
    temp_dic['ownerCode'] = df['CONSIGN_SCC']
    temp_dic['ownerName'] = df['CONSIGN_NAME']
    temp_dic['ownerCodeCompare'] = df['CONSIGN_SCC_COMPARE']
    temp_dic['ownerNameCompare'] = df['CONSIGN_NAME_COMPARE']
    for key, value in temp_dic.items():
        str += "\"%s\":\"%s\"," % (key, value)
    str = str[:-1]
    return str + "},"
def deal_comparison_dic(df):
    temp_dic = {}
    str = "{"
    temp_dic['code_t'] = df['G_NAME']
    temp_dic['qty1'] = df['G_QTY']
    temp_dic['originCountry'] = df['ORIGIN_COUNTRY']
    temp_dic['codeTCompare'] = df['G_NAME_COMPARE']
    temp_dic['qty1Compare'] = df['G_QTY_COMPARE']
    temp_dic['originCountryCompare'] = df['ORIGIN_COUNTRY_COMPARE']
    for key, value in temp_dic.items():
        str += "\"%s\":\"%s\"," % (key, value)
    str = str[:-1]
    return str + "},"